In [ ]:
from pathlib import Path

datapath = Path("data")

# _Slippy_ maps (Mapas "escorregantes")


<div class="alert-info">

### Perguntas
- O que são e quando usar?
- Quais bibliotecas são mais adequadas para "o meu caso?"

### Objetivos
- Exibir dados em mapas _slippy_

</div>


Mapas _slippy_ são uma ferramenta poderosa para exibir dados complexos e permitir ao usuário final uma navegação/exploração de alto nível.

In [ ]:
from toolz.dicttoolz import valmap

invert = lambda x: x[::-1]

pybrs = {
    "Campinas": [-47.060833, -22.905833],
    "Brasília": [-47.882778, -15.793889],
    "Joinville": [-48.845833, -26.303889],
    "Rio de Janeiro": [-43.196389, -22.908333],
    "Caxias do Sul": [-51.178889, -29.167778],
    "Curitiba": [-49.271944, -25.429722],
    "São Paulo": [-46.633333, -23.55],
    "Cidade Nova": [-43.2025, -22.91],
    "Brasília": [-47.882778, -15.793889],
    "Porto de Galinhas": [-35.0011, -8.5041],
    "São José dos Campos": [-45.886944, -23.178889],
    "Florianópolis": [-48.548889, -27.596944],
    "Belo Horizonte": [-43.933333, -19.916667],
    "Ribeirão Preto": [-47.81, -21.177778],
}

pybrs = valmap(invert, pybrs)

### folium

In [ ]:
import folium
from folium.plugins import Fullscreen


m = folium.Map(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}",
    attr="ESRI",
    zoom_start=5,
)

for city, location in pybrs.items():
    folium.Marker(location=location).add_to(m)
    
m.fit_bounds(m.get_bounds())
m

<div class="alert-warning">

Exercício:

1. Adicione o nome da cidade como popup.
2. explore os `plugins` e adicione algo que facilitaria a navegação dos dados.

Dica: olhe os `kw` da classe `Marker`.

</div>

<div class="alert-success">

</div>

```python
import folium.plugins


"""..."""
```

### Exemplo completo dados raster, vetoriais para um mapa temático

In [ ]:
import folium

lon, lat = -86.276, 30.935 

m = folium.Map(
    location=[lat, lon],
    tiles="Cartodb Positron",
    zoom_start=5
)

In [ ]:
from pathlib import Path
import json

path = Path("data")

f = path.joinpath("stations.geojson").open()
stations = json.load(f)

f = path.joinpath("hfradar.geojson").open()
hfradar = json.load(f)

In [ ]:
icon_size = (14, 14)

for feature in stations["features"]:
    lon, lat = feature["geometry"]["coordinates"]
    icon_url = feature["properties"]["icon"]
    popup = feature["properties"]["popupcontent"]

    icon = folium.features.CustomIcon(
        icon_url,
        icon_size=(14, 14)
    )

    marker = folium.map.Marker(
        location=[lat, lon],
        icon=icon,
        popup=folium.map.Popup(popup)
    )
    m.add_child(marker)

In [ ]:
for feature in hfradar["features"]:
    if feature["geometry"]["type"] == "Point":
        lon, lat = feature["geometry"]["coordinates"]
        icon_url = feature["properties"]["icon"]
        popup = feature["properties"]["popupcontent"]

        icon = folium.features.CustomIcon(
            icon_url,
            icon_size=(14, 14)
        )
        marker = folium.Marker(
            location=[lat, lon],
            icon=icon,
            popup=folium.Popup(popup)
        )
        m.add_child(marker)
    elif feature["geometry"]["type"] == "Polygon":
        gjson = folium.features.GeoJson(feature)
        m.add_child(gjson)

In [ ]:
m

In [ ]:
import pandas as pd


url = (
    "https://whereonmars.carto.com/api/v2/sql?"
    "filename=themartian&"
    "q=SELECT+*+FROM+(select+*+from+public.themartian)+as+subq+&"
    "format=csv&"
    "bounds=&"
    "api_key=&"
    "skipfields=the_geom_webmercator"
)

df = pd.read_csv(url, index_col="cartodb_id").sort_index().drop("the_geom", axis=1)

In [ ]:
import folium
from folium.plugins import Draw, Fullscreen

opm_attr = '<a href="https://github.com/openplanetary/opm/wiki/OPM-Basemaps" target="blank">OpenPlanetaryMap</a>'

m = folium.Map(
    attr=f"NASA/MOLA |{opm_attr}",
    max_zoom=5,
    tiles="https://s3-eu-west-1.amazonaws.com/whereonmars.cartodb.net/mola-color/{z}/{x}/{-y}.png"
)

Fullscreen().add_to(m);

In [ ]:
kw = {
    "fill": True,
    "color": "white",
    "fill_color": "white",
    "fill_opacity": 1,
}

for k, row in df.iterrows():
    title = row["title"] if isinstance(row["title"], str) else None
    location = [row["lat"], row["lng"]]
    if title:
        folium.CircleMarker(location=location, popup=title, radius=5, **kw).add_to(m)
    else:
        folium.CircleMarker(location=location, radius=1, **kw).add_to(m)

In [ ]:
m

### ipyleaflet

In [ ]:
from ipyleaflet import Map, Velocity, TileLayer, basemaps
import xarray as xr

zoom = 3

m = Map(
    center=[44, -130],
    zoom=3,
    interpolation="nearest",
    basemap=basemaps.CartoDB.DarkMatter,
)

In [ ]:
ds = xr.open_dataset(datapath.joinpath("wind-global.nc"))

display_options = {
    "velocityType": "Global Wind",
    "displayPosition": "bottomleft",
    "displayEmptyString": "No wind data"
}
wind = Velocity(
    data=ds, 
    zonal_speed="u_wind", meridional_speed="v_wind", 
    latitude_dimension="lat", longitude_dimension="lon", 
    velocity_scale=0.01, max_velocity=20, 
    display_options=display_options
)
m.add_layer(wind)

In [ ]:
m

<div class="alert-success">

_Slippy maps_ são poderosas ferramentas de visualização que permitem "compactar" muita informação e entregar ao usuário final o poder de explorar conjunto de dados complexos.

* folium é ideal para mapas que serão "servidos" ou que fazem uso dos diversos plugins do leaflet.
* ipyleaflet é ideal para interações de duas vias entre o mapa (JS) e Python.

</div>